In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('../data/chembl_32.db')

In [3]:
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
try:
    assert len(cursor.fetchall()) > 1
except AssertionError:
    print('Incorrect database. Please download the database again.')

In [6]:
_sql = """
SELECT
    MOLECULE_DICTIONARY.pref_name,
    MOLECULE_DICTIONARY.CHEMBL_ID as chembl_idx,
    COMPOUND_STRUCTURES.canonical_smiles as smiles,
    ASSAYS.chembl_id as assay_id,
    ACTIVITIES.pchembl_value as pchembl_value,
    GO_CLASSIFICATION.go_id as go_id,
    GO_CLASSIFICATION.pref_name as go_term
FROM MOLECULE_DICTIONARY
JOIN ACTIVITIES ON MOLECULE_DICTIONARY.molregno == ACTIVITIES.molregno
JOIN ASSAYS ON ACTIVITIES.assay_id == ASSAYS.assay_id
JOIN TARGET_DICTIONARY on ASSAYS.tid == TARGET_DICTIONARY.tid
JOIN TARGET_COMPONENTS on TARGET_DICTIONARY.tid == TARGET_COMPONENTS.tid
JOIN COMPONENT_SEQUENCES on TARGET_COMPONENTS.component_id == COMPONENT_SEQUENCES.component_id
JOIN COMPONENT_GO on COMPONENT_SEQUENCES.component_id == COMPONENT_GO.component_id
JOIN GO_CLASSIFICATION on COMPONENT_GO.go_id == GO_CLASSIFICATION.go_id
JOIN COMPOUND_STRUCTURES on MOLECULE_DICTIONARY.molregno == COMPOUND_STRUCTURES.molregno
WHERE
    ASSAYS.tid is not null
    and ASSAYS.assay_organism = 'Homo sapiens'
"""

In [7]:
df = pd.read_sql(_sql, con=conn)
df.head()

,pref_name,chembl_idx,smiles,assay_id,pchembl_value,go_id,go_term
0,None,CHEMBL344142,C[C@]12CCC(=O)C=C1C(=O)CC1C2CC[C@@]2(C)C1CC[C@...,CHEMBL615157,NaN,GO:0000166,nucleotide binding
1,None,CHEMBL344142,C[C@]12CCC(=O)C=C1C(=O)CC1C2CC[C@@]2(C)C1CC[C@...,CHEMBL615157,NaN,GO:0016491,oxidoreductase activity
2,SPRIOLACTONE,CHEMBL400534,C[C@]12CCC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@@]2(C)[...,CHEMBL615157,NaN,GO:0000166,nucleotide binding
3,SPRIOLACTONE,CHEMBL400534,C[C@]12CCC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@@]2(C)[...,CHEMBL615157,NaN,GO:0016491,oxidoreductase activity
4,None,CHEMBL3137945,C[C@]12CC[C@@]3(CCC(=O)O3)CC1CC[C@@H]1[C@@H]2C...,CHEMBL615157,NaN,GO:0000166,nucleotide binding


In [8]:
df.to_csv('../data/chembl_go.tsv', sep='\t', index=False)